Age
Total Spend
Items Purchased
Average Rating
Discount Applied
Days Since Last Purchase
Satisfaction Level
City
Gender
Membership Type

In [16]:
import os
import glob
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
main_dir = 'data/processed/'
file_name = 'cluster_*.csv'
file_path = os.path.join(main_dir, file_name)

In [26]:
# Use glob to find files starting with 'cluster' and ending with '.csv'
csv_files = glob.glob(file_path)

In [27]:
# Read each CSV into a DataFrame and store them in a list
dataframes = [pd.read_csv(file) for file in csv_files]

# Recommendations

In [31]:
# Dictionary to store the cosine similarity DataFrames for each DataFrame in the list
cosine_sim_dfs = {}

In [32]:
for i, df in enumerate(dataframes):
    # Extract features
    df_features = df[df.columns]
    
    # cosine similarity
    cosine_sim = cosine_similarity(df_features)
    
    # Convert similarity matrix into a DataFrame
    cosine_sim_df = pd.DataFrame(cosine_sim, index=df.index, columns=df.index)
    
    # Store the similarity DataFrame with a key like 'df_1', 'df_2', etc.
    cosine_sim_dfs[f'df_{i+1}'] = cosine_sim_df

In [40]:
cosine_sim_dfs['df_1'].head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,1.000000,0.998534,0.999034,0.996634,0.999284,0.999232,0.999357,0.998666,0.996634,0.999284,...,0.995384,0.999253,0.994916,0.999476,0.994381,0.998615,0.993773,0.998237,0.993100,0.998268
1,0.998534,1.000000,0.995898,0.997987,0.999057,0.997429,0.999063,0.998004,0.997987,0.999057,...,0.996121,0.998009,0.995400,0.998411,0.994830,0.996795,0.993970,0.996597,0.993262,0.996305
2,0.999034,0.995898,1.000000,0.994040,0.997021,0.998747,0.998153,0.996825,0.994040,0.997021,...,0.993904,0.998286,0.993615,0.998270,0.993329,0.998516,0.992899,0.997898,0.992473,0.998385
3,0.996634,0.997987,0.994040,1.000000,0.995673,0.997823,0.997091,0.993099,1.000000,0.995673,...,0.999044,0.993992,0.998677,0.996277,0.998170,0.993664,0.997662,0.995339,0.997016,0.993395
4,0.999284,0.999057,0.997021,0.995673,1.000000,0.997414,0.998776,0.999636,0.995673,1.000000,...,0.993409,0.999328,0.992652,0.999002,0.991975,0.997803,0.991080,0.996881,0.990267,0.997235


In [ ]:
# Define a recommendation function to work for a specific DataFrame
def get_recommendations(dataframe_key, user_id, top_n=3):
    # Retrieve the appropriate similarity matrix
    sim_df = cosine_sim_dfs[dataframe_key]
    
    # Get similarity scores for the given user
    sim_scores = sim_df[user_id]
    
    # Sort users based on similarity score (highest to lowest), skipping the user themselves
    similar_users = sim_scores.sort_values(ascending=False)[1:top_n+1]
    
    return similar_users

In [ ]:
# Example: Get recommendations for user_id=0 from the first DataFrame
print(get_recommendations('df_1', user_id=0, top_n=3))